Web Scraper for exported data

Made by Artem Yushko

In [5]:
# the God's library, just like regex for data cleaners
!pip install beautifulsoup4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
# all the imports we will need
from bs4 import BeautifulSoup
import requests
import math
import os

In [ ]:
# retrieving the page links

# reading the Telegram channel exports
html_docs = []

for filename in os.listdir('/content/drive/MyDrive/artem-yushko/data-artem/telegram-exports/Suspilne/'):
   with open(os.path.join('/content/drive/MyDrive/artem-yushko/data-artem/telegram-exports/Suspilne/', filename), 'r') as f:
       msg = f.read()
       html_docs.append(msg)       

# extracting the links from there
urls = []

for doc in html_docs:
    soup = BeautifulSoup(doc, 'html.parser')
    # getting all the links and moving them to a specific list
    for link in soup.find_all('a'):
        l = link.get('href')
        urls.append(l)

# making sure that the links left are only the urls we want
urls = [url for url in urls if '//suspilne.media' in url]
urls = [url for url in urls if not '@' in url]

# removing the duplicate links
urls = list(set(urls))

print("Number of URLs to parse: " + str(len(urls)))

In [ ]:
# set for the lines to be added
lines = []

# different counters for statistics stuff
url_counter = 0
result_counter = 0

# for url in the list of generater urls
for url in urls:
    # statistics
    if url_counter % 300 == 0:
        print(str(math.floor((url_counter/len(urls))*100)) + ' % of the urls are processed. ' + str(result_counter) + ' lines were added.')
    # following the link
    # r = requests.get(url)
    r = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}) # uncomment if 403 or 429 emerge way too often
    url_counter = url_counter + 1
    if r.status_code == 403 or r.status_code == 429:
        print("The website thinks you're DDOSing it!")
    # if the link is valid
    if r.status_code == 200:
        # get the content
        p = r.content
        # most of the stuff we need comes in <p> tags, so we'll focus on them
        soup_p = BeautifulSoup(p, 'html5lib')
        text = soup_p.find_all('p')
        # for each text object we obtained
        for s in text:
            # splitting them into lines (not sentences)
            t = s.get_text().split('\n')
            # if the sentence is not empty and finishes on a period (a nice way to catch the artifacts)
            if t[0] and t[0][-1] == '.':
                # then append it to the list
                lines.append(t[0])
                result_counter = result_counter + 1
    else:
      print(f"Unexpected error! This is the status code: {r.status_code}")

In [ ]:
print(len(lines))
print(lines[7])

84002
Нардеп Тарас Козак перебуває в Росії, а нардеп Віктор Медведчук, у якого у вівторок провели обшуки, знаходиться в Україні.


In [ ]:
text = '\n'.join(lines)
output_path = "/content/drive/MyDrive/artem-yushko/data-artem/raw/Suspilne.txt"
with open(output_path, 'w') as f:
  f.write(text)

import re
print(len(re.findall('\.', text)))

229423
